## Downscaling with the DeepESD model

This notebook showcases a simple application of deep4downscaling for the statistical downscaling of precipitation. To do so, we will implement the following actions:

- Define and train the DeepESD architecture [1].
- Downscale and evaluate results over a test period.
- Downscale outputs from a Global Climate Model (GCM).
- Generate the corresponding downscaled climate change signals.

### Train the model

In [ ]:
DATA_PATH = './data/input'
FIGURES_PATH = './figures'
MODELS_PATH = './models'
ASYM_PATH = './data/asym'

When working with climate data, xarray is an essential library, and deep4downscaling heavily relies on it. For the deep learning component, deep4downscaling uses PyTorch, one of the most popular frameworks in the field.

In [ ]:
import xarray as xr
import torch
from torch.utils.data import DataLoader, random_split

import sys; sys.path.append('/home/jovyan/deep4downscaling')
import deep4downscaling.viz
import deep4downscaling.trans
import deep4downscaling.deep.loss
import deep4downscaling.deep.utils
import deep4downscaling.deep.models
import deep4downscaling.deep.train
import deep4downscaling.deep.pred
import deep4downscaling.metrics
import deep4downscaling.metrics_ccs

We will begin by loading the predictor. In this case, we select various large-scale variables from ERA5 at different height levels. These variables are already stored in a NetCDF file, the standard data format for deep4downscaling. Unfortunately, due to GitHub's size restrictions, we are unable to upload these files to the repository. However, the following cells provide an overview of the data, making it straightforward to reproduce this notebook with a similar file.

In [ ]:
# Load predictors
predictor_filename = f'{DATA_PATH}/ERA5_NorthAtlanticRegion_1-5dg_full.nc'
predictor = xr.open_dataset(predictor_filename)

In [ ]:
predictor

The deep4downscaling library provides several functions to facilitate an initial visualization of the data. For example, the `deep4downscaling.viz.multiple_map_plot` function allows you to visualize an `xarray.Dataset`. These functions rely on matplotlib and cartopy. By default, the figure is saved as a `.pdf` file in the path specified by the `output_path argument`.

In [ ]:
deep4downscaling.viz.multiple_map_plot(data=predictor.mean('time'),
                                       output_path=f'./{FIGURES_PATH}/predictor_climatology.pdf')

The predictand is an `xarray.Dataset` containing a single variable (the target). In this notebook, we will focus on downscaling accumulated precipitation over the region of peninsular Spain and the Balearic Islands.

In [ ]:
predictand_filename = f'{DATA_PATH}/pr_AEMET.nc'
predictand = xr.open_dataset(predictand_filename)

In [ ]:
predictand

Similar to the predictors, deep4downscaling can also be used for an initial visualization of the predictand.

In [ ]:
day_to_viz = '10-04-1981'
deep4downscaling.viz.simple_map_plot(data=predictand.sel(time=day_to_viz),
                                     colorbar='hot_r', var_to_plot='pr',
                                     output_path=f'./{FIGURES_PATH}/predictand_day.pdf')

Deep4downscaling also includes several common preprocessing techniques used in statistical downscaling, such as removing NaN values, aligning datasets (e.g., across time), bias adjustment, and standardization, among others.

In [ ]:
# Remove days with nans in the predictor
predictor = deep4downscaling.trans.remove_days_with_nans(predictor)

# Align both datasets in time
predictor, predictand = deep4downscaling.trans.align_datasets(predictor, predictand, 'time')

To adhere to the standard training/validation scheme in the machine learning field, we divide the predictors and predictand into training and test sets.

In [ ]:
years_train = ('1980', '1981')
years_test = ('1982', '1983')

x_train = predictor.sel(time=slice(*years_train))
y_train = predictand.sel(time=slice(*years_train))

x_test = predictor.sel(time=slice(*years_test))
y_test = predictand.sel(time=slice(*years_test))

Before feeding the predictors to the deep learning model, we standardize them to have a mean of zero and a standard deviation of one. This is done using the `deep4downscaling.trans.standardize` function.

In [ ]:
x_train_stand = deep4downscaling.trans.standardize(data_ref=x_train, data=x_train)

For training and inference, the data will be transformed into the torch.Tensor type. To facilitate the transition from NetCDF to torch.Tensor, especially when computing projections (predictions), we define a mask around the predictand to use throughout the entire workflow.

In [ ]:
y_mask = deep4downscaling.trans.compute_valid_mask(y_train) 

All deep learning models implemented in deep4downscaling flatten their output into a vector, standardizing its dimensions to the shape `(time, grid point)`.

In [ ]:
y_train_stack = y_train.stack(gridpoint=('lat', 'lon'))
y_mask_stack = y_mask.stack(gridpoint=('lat', 'lon'))

The DeepESD architecture consists of a set of convolutional layers followed by a final dense layer. In our case, since the predictand contains NaN values for sea grid points, we filter out these grid points to save computation. This reduces the number of neurons in the final fully connected layer. By applying this operation using the mask, the conversion between the model's output and the corresponding NetCDF becomes straightforward.

In [ ]:
y_mask_stack_filt = y_mask_stack.where(y_mask_stack==1, drop=True)
y_train_stack_filt = y_train_stack.where(y_train_stack['gridpoint'] == y_mask_stack_filt['gridpoint'],
                                             drop=True)

The deep4downscaling library includes various loss functions for training deep learning models. In this notebook, we follow [2] and focus on the ASYmmetric loss function (ASYM). We have provided the values asym_weight=3 and cdf_weight=10 as an example of the flexibility of the loss function. Default values of asym_weight=1 and cdf_weight=2 are equivalent to the original loss.Asym at [3].Implementing custom loss functions should be straightforward, as they follow the typical PyTorch conventions.

In [ ]:
loss_function = deep4downscaling.deep.loss.Asym(ignore_nans=True,
                                                asym_path=ASYM_PATH)

For this loss function to work, we need to pre-compute a gamma distribution for each grid point in the predictand data (training set) on a yearly basis and calculate the mean of their parameters (see [3] for more details). This process can be handled by deep4downscaling.

In [ ]:
if loss_function.parameters_exist():
    loss_function.load_parameters()
else:
    loss_function.compute_parameters(data=y_train_stack_filt,
                                     var_target='pr')

NetCDF is not well-suited for use with PyTorch (or for converting to the `torch.Tensor` type). In contrast, NumPy is.

In [ ]:
x_train_stand_arr = deep4downscaling.trans.xarray_to_numpy(x_train_stand)
y_train_arr = deep4downscaling.trans.xarray_to_numpy(y_train_stack_filt)

With our data now in the numpy format, we can create the `torch.Dataset` and `torch.DataLoader` to feed batches of data to the deep learning model during training.

In [ ]:
# Create Dataset
train_dataset = deep4downscaling.deep.utils.StandardDataset(x=x_train_stand_arr,
                                                            y=y_train_arr)

# Split into training and validation sets
train_dataset, valid_dataset = random_split(train_dataset,
                                            [0.9, 0.1])

# Create DataLoaders
batch_size = 64

train_dataloader = DataLoader(train_dataset, batch_size=batch_size,
                              shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size,
                              shuffle=True)

Deep4downscaling includes several predefined deep learning architectures (e.g., DeepESD and U-Net), but custom architectures can be easily defined using the standard PyTorch framework. However, because deep4downscaling relies on a final flattening operation (as mentioned earlier), we recommend reviewing the implementations in `deep4downscaling.deep.models` and using them as a foundation.

While deep4downscaling lacks a formal documentation page, all its functions and arguments are properly documented within the code.

In [ ]:
?deep4downscaling.deep.models.DeepESDpr

In this notebook, we will train the DeepESD architecture with a single final convolutional layer.

In [ ]:
model_name = 'deepesd_pr'
model = deep4downscaling.deep.models.DeepESDpr(x_shape=x_train_stand_arr.shape,
                                               y_shape=y_train_arr.shape,
                                               filters_last_conv=1,
                                               stochastic=False)

We set the typical training hyperparameters, as is commonly done in PyTorch.

In [ ]:
num_epochs = 10000
patience_early_stopping = 20

learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(),
                             lr=learning_rate)

Deep learning models can run on either CPU or GPU devices. We provide the corresponding `.yml` environment files (`deep4downscaling/requirement`) to set up a basic Conda environment for running deep4downscaling.

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

# Move ASYM paramters to device
loss_function.prepare_parameters(device=device)

Deep4downscaling provides the `deep4downscaling.deep.train.standard_training_loop`, which implements a basic training routine. Models are saved based on their performance on a validation set through an early stopping process, with the final saved model being the one that achieves the best score on this set. To disable early stopping, you can pass `None` to the `patience_early_stopping` argument. We recommend users consult the `?deep4downscaling.deep.train.standard_training_loop` for further details about this function.

In [ ]:
train_loss, val_loss = deep4downscaling.deep.train.standard_training_loop(
                            model=model, model_name=model_name, model_path=MODELS_PATH,
                            device=device, num_epochs=num_epochs,
                            loss_function=loss_function, optimizer=optimizer,
                            train_data=train_dataloader, valid_data=valid_dataloader,
                            patience_early_stopping=patience_early_stopping)

### Downscale the test set

Once a model has been trained and saved as a `.pt` file, it is easy to compute predictions on a new set of predictors. In this example, we will compute predictions on the test set, which was subset a few cells above. It is important to standardize the test data using the mean and standard deviation computed from the training set.

In [ ]:
# Load the model weights into the DeepESD architecture
model.load_state_dict(torch.load(f'{MODELS_PATH}/{model_name}.pt'))

# Standardize
x_test_stand = deep4downscaling.trans.standardize(data_ref=x_train, data=x_test)

# Compute predictions
pred_test = deep4downscaling.deep.pred.compute_preds_standard(
                                x_data=x_test_stand, model=model,
                                device=device, var_target='pr',
                                mask=y_mask, batch_size=16)

In [ ]:
# Visualize the predictions
deep4downscaling.viz.simple_map_plot(data=pred_test.mean('time'),
                                     colorbar='hot_r', var_to_plot='pr',
                                     output_path=f'./{FIGURES_PATH}/prediction_test_mean.pdf')

The `deep4downscaling.metrics` module, included within deep4downscaling, implements various metrics commonly used to assess deep learning models in the context of statistical downscaling. These include biases of different indices, spatial and probabilistic metrics, and multivariate indices, among others. In this example, we demonstrate its use by computing the relative bias of the Rx1day index between the target (test set) and the predictions for the winter months.

In [ ]:
bias_rel_rx1day = deep4downscaling.metrics.bias_rel_rx1day(target=y_test, pred=pred_test,
                                                           var_target='pr', season='winter') 

In [ ]:
print(bias_rel_rx1day)